# A Replay of US Presidential Election outcomes: calculating a portfolio's sensitivity to a Democratic or Republican presidential election win.

In this example, we will create custom factor shocks scenarios that showcase the potential impact of a Democratic or
Republican presidential election win on the performance of your portfolio. The objective is to build custom factor shocks
that encapsulate equity factors' performance during each US presidential election outcome. These factor shocks will be
the average of factor moves on each outcome over the previous 6 presidential elections (2000, 2004, 2008, 2012, 2016, 2020).

These are the steps to build the factor shocks:
1. Identify the time period and the outcome of each presidential election for the previous 6 elections
2. Select a risk model and capture the market conditions one week post the election results announcement, which entails pulling and aggregating realized factor returns over the one-week period post the presidential election results announcement.
3. The final factor shocks are an average of factor moves on each election outcome.

## Step1: Select the risk model

Select a risk model. See [full list of available risk models here](https://marquee.gs.com/s/discover/data-services/catalog?Category=Factor+Risk+Model).

In [2]:
from gs_quant.session import GsSession, Environment
from gs_quant.markets.scenario import FactorScenario, FactorShock, FactorShockParameters, FactorScenarioType
from gs_quant.markets.portfolio_manager import PortfolioManager, ScenarioCalculationMeasure
from gs_quant.models.risk_model import FactorRiskModel
import datetime as dt
import pandas as pd
import numpy as np
from time import sleep

GsSession.use(Environment.PROD)

risk_model_id = "BARRA_USFAST"
risk_model = FactorRiskModel.get(risk_model_id)

## Step 2: Identify time periods for each election outcome.

Democrats won elections in 2008, 2012, and 2020 whereas Republicans won elections in 2000, 2004 and 2016

1. **Nov 7, 2000 to Nov 14, 2000**: Republican Party win
2. **Nov 2, 2004 ot Nov 9, 2004**: Republican Party win
3. **Nov 4, 2008 to Nov 11, 2008**: Democratic Party win
4. **Nov 6, 2012 to Nov 13, 2012**: Democratic Party win
5. **Nov 8, 2016 to Nov 15, 2016**: Republican Party win
6. **Nov 3, 2020 to Nov 10, 2020**: Democratic Party win.

In [3]:
presidential_elections_2000 = ((dt.date(2000, 11, 7)), dt.date(2000, 11, 14))
presidential_elections_2004 = ((dt.date(2004, 11, 2)), dt.date(2004, 11, 9))
presidential_elections_2008 = ((dt.date(2008, 11, 4)), dt.date(2008, 11, 11))
presidential_elections_2012 = ((dt.date(2012, 11, 6)), dt.date(2012, 11, 13))
presidential_elections_2016 = ((dt.date(2016, 11, 8)), dt.date(2016, 11, 15))
presidential_elections_2020 = ((dt.date(2020, 11, 3)), dt.date(2020, 11, 10))

democrats_win_dates = [presidential_elections_2008, presidential_elections_2012, presidential_elections_2020]
republicans_win_dates = [presidential_elections_2000, presidential_elections_2004, presidential_elections_2016]

## Step 3: What were the factor moves in the week following US presidential elections?

We then pull realized daily factor returns for each one-week election period and aggregate them to get factor performance over the period.

In [4]:
election_period_to_factor_returns_map = {}
for period in democrats_win_dates + republicans_win_dates:
    start, end = period
    factor_returns = risk_model.get_factor_returns_by_name(start, end)

    factor_returns.index = pd.to_datetime(factor_returns.index.values, format="%Y-%m-%d")
    factor_returns = factor_returns.sort_index()

    # aggregate returns
    returns_geometrically_aggregated = np.multiply.reduce(1 + factor_returns.values / 100) - 1
    factor_shocks_df = pd.DataFrame(
        returns_geometrically_aggregated, index=factor_returns.columns.values, columns=["factorShocks"]
    )

    election_period_to_factor_returns_map[period] = factor_shocks_df

## Step 4: Build the factor shocks.

The final factor shocks will be the average of the performance of factors for all periods.

In [5]:
factor_shocks_democratic_win_df = pd.concat(
    [election_period_to_factor_returns_map.get(period) for period in democrats_win_dates], axis=1
)
factor_shocks_republican_win_df = pd.concat(
    [election_period_to_factor_returns_map.get(period) for period in republicans_win_dates], axis=1
)

number_of_democratic_wins = factor_shocks_democratic_win_df.columns.size
number_of_republican_wins = factor_shocks_republican_win_df.columns.size

# Filter factors that have been demised (your portfolio won't have exposures to them anyway)
available_factors = risk_model.get_factor_data()


unavailable_factors_dem_wins = set(factor_shocks_democratic_win_df.index.tolist()) - set(
    available_factors['name'].tolist()
)
factor_shocks_democratic_win_df = factor_shocks_democratic_win_df.drop(index=list(unavailable_factors_dem_wins))

unavailable_factors_rep_wins = set(factor_shocks_republican_win_df.index.tolist()) - set(
    available_factors['name'].tolist()
)
factor_shocks_republican_win_df = factor_shocks_republican_win_df.drop(index=list(unavailable_factors_rep_wins))

factor_shocks_democratic_win_df = (
    factor_shocks_democratic_win_df.apply(np.sum, axis=1).to_frame() / number_of_democratic_wins
) * 100
factor_shocks_republican_win_df = (
    factor_shocks_republican_win_df.apply(np.sum, axis=1).to_frame() / number_of_republican_wins
) * 100

## Step 4: Create the custom shock scenario

Once we have factor shocks, it is time to create the custom shock scenario. We will have:

1. A `FactorShockParameters` which comprises the factor shocks constructed above as well as the risk model used and whether we are propagating shocks.
2. The type of the scenario with is `FactorScenarioType.Factor_Shock`
3. Other scenario metadata such as name and description

In [6]:
dems_shocks_map = factor_shocks_democratic_win_df.to_dict(orient="split")

rep_shocks_map = factor_shocks_republican_win_df.to_dict(orient="split")

final_factor_shocks_democratic_wins = [
    FactorShock(factor=f, shock=v)
    for f, v in dict(zip(dems_shocks_map.get('index'), [s[0] for s in dems_shocks_map.get('data')])).items()
]
final_factor_shocks_republican_wins = [
    FactorShock(factor=f, shock=v)
    for f, v in dict(zip(rep_shocks_map.get('index'), [s[0] for s in rep_shocks_map.get('data')])).items()
]

# Since we are shocking every single factor; no need to propagate shocks
parameters_democratic_win = FactorShockParameters(
    risk_model=risk_model_id, propagate_shocks=False, factor_shocks=final_factor_shocks_democratic_wins
)


parameters_republican_win = FactorShockParameters(
    risk_model=risk_model_id, propagate_shocks=False, factor_shocks=final_factor_shocks_republican_wins
)

democratic_election_win_scenario = FactorScenario(
    name="Democratic Presidential Wins 1 week after (2008, 2012, 2020)",
    description=f"Factor moves in {risk_model_id} one week post Democratic presidential election win (2008, 2012, 2020). Factor shocks are an average of aggregate factor returns on each period",
    type=FactorScenarioType.Factor_Shock,
    parameters=parameters_democratic_win,
    tags=["Presidential Election"],
)

democratic_election_win_scenario.save()
republican_election_win_scenario = FactorScenario(
    name="Republican Presidential Wins 1 week after (2000, 2004, 2016)",
    description=f"Factor moves in {risk_model_id} one week post Republican presidential election win (2000, 2004, 2016). Factor shocks are an average of aggregate factor returns on each period",
    type=FactorScenarioType.Factor_Shock,
    parameters=parameters_republican_win,
    tags=["Presidential Election"],
)

republican_election_win_scenario.save()

sleep(2)

## Step 5: Run both election outcome stress tests on your portfolio to observe estimated PnL impact

To run the scenarios on a portfolio, we will need the following:

- Scenarios: The Democratic election win and Republican election win scenarios
- A date: date on which to run the two scenarios. By default, the date is the latest date with risk analytics are available.
- Risk Model: The selected risk model.
- Measures: Metrics to return, which include:
    1. Total portfolio estimated factor PnL
    2. Total portfolio PnL by GICS sector & industry
    3. Total portfolio PnL by region
    4. Total portfolio PnL by the direction (LONG/SHORT) of your portfolio holdings,
    5. Estimated performance of each asset in your portfolio.

In [7]:
portfolio_id = "PORTFOLIO ID"
pm = PortfolioManager(portfolio_id)
risk_report_latest_end_date = pm.get_factor_risk_report(risk_model_id).latest_end_date
election_outcome_scenario_analytics = pm.get_factor_scenario_analytics(
    scenarios=[republican_election_win_scenario, democratic_election_win_scenario],
    date=risk_report_latest_end_date,
    measures=[
        ScenarioCalculationMeasure.SUMMARY,
        ScenarioCalculationMeasure.ESTIMATED_FACTOR_PNL,
        ScenarioCalculationMeasure.ESTIMATED_PNL_BY_SECTOR,
        ScenarioCalculationMeasure.ESTIMATED_PNL_BY_REGION,
        ScenarioCalculationMeasure.ESTIMATED_PNL_BY_DIRECTION,
    ],
    risk_model=risk_model_id,
)